In [ ]:
# =====================
# 1) Gerekli kütüphaneleri yükle
# =====================
!pip install transformers datasets accelerate -q

import pandas as pd
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModel, Trainer, TrainingArguments
import torch.nn as nn

# GPU kontrolü
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Kullanılan cihaz:", device)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.0 MB/s eta 0:00:00
Kullanılan cihaz: cuda


In [ ]:
# =====================
# 2) CSV dosyasını yükle
# =====================
# Colab'a dosya yükleme
from google.colab import files
uploaded = files.upload()

# Yüklediğin dosya ismini buraya yaz
csv_dosya = "veri_seti.csv"

df = pd.read_csv(csv_dosya)

df = df[["text", "total_score"]]


df["total_score"] = df["total_score"].astype(float)
df["total_score"] = df["total_score"] / 10



df = df.dropna(subset=["total_score"])


print(df[["total_score"]].isna().sum())
print(df.describe())


df.head()

Saving veri_seti.csv to veri_seti.csv
total_score    0
dtype: int64
       total_score
count   534.000000
mean      0.694757
std       0.270329
min       0.000000
25%       0.400000
50%       0.800000
75%       0.900000
max       1.000000


,text,total_score
0,hatırlamanızı isterim. Bugün sizin kadar şansl...,0.9
1,Küçük bir çocuk size gelse ve en büyük günahın...,0.9
2,"Lilit… Ademle eşit yaratılan, Adem’in üstünlüğ...",0.8
3,rengarenk karakterlere ve dolayısıyla eşsiz dü...,0.9
4,Turk-101 Section :20\nBaşak Berna Cordan\n25.1...,0.4


In [ ]:
# =====================
# 3) Dataset sınıfı
# =====================
class TextScoreDataset(Dataset):
    def __init__(self, texts, scores, tokenizer, max_len=256):
        self.texts = texts
        self.scores = scores
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
      text = str(self.texts[idx])
      targets = torch.tensor(self.scores[idx], dtype=torch.float)

      encoding = self.tokenizer(
          text,
          truncation=True,
          padding="max_length",
          max_length=self.max_len,
          return_tensors="pt"
      )

      output = {
          "input_ids": encoding["input_ids"].squeeze(0),
          "attention_mask": encoding["attention_mask"].squeeze(0),
          "labels": targets
      }

      #print(f"Sample {idx} output keys: {list(output.keys())}")
      return output




In [ ]:
class BertSingleOutput(nn.Module):
    def __init__(self, model_name):
        super(BertSingleOutput, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)
        self.linear = nn.Linear(self.bert.config.hidden_size, 1)  # tek çıktı

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0]  # CLS token
        pooled_output = self.dropout(pooled_output)
        logits = self.linear(pooled_output)

        loss = None
        if labels is not None:
            loss_fn = nn.MSELoss()
            loss = loss_fn(logits.view(-1), labels.view(-1))  # shape uyumu için düzleştir

        return {"loss": loss, "logits": logits}

from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")
model_name = "dbmdz/bert-base-turkish-cased"

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

In [ ]:

scores = df["total_score"].values

train_size = int(0.8 * len(df))
val_size = len(df) - train_size

train_texts = df["text"].values[:train_size]
train_scores = scores[:train_size]

val_texts = df["text"].values[train_size:]
val_scores = scores[train_size:]

train_dataset = TextScoreDataset(train_texts, train_scores, tokenizer)
val_dataset = TextScoreDataset(val_texts, val_scores, tokenizer)

model = BertSingleOutput(model_name).to(device)


model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

In [ ]:

from transformers import Trainer

training_args = TrainingArguments(
    output_dir="./bert_turkish_model",
    num_train_epochs=20,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    load_best_model_at_end=True,
    report_to="none"  # wandb kapalı
)


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    mse = ((logits.squeeze() - labels.squeeze()) ** 2).mean()
    return {"mse_total_score": mse}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss,Mse Total Score
1,No log,0.136585,0.136585
2,No log,0.311568,0.311568
3,No log,0.064782,0.064782
4,No log,0.138664,0.138664
5,No log,0.127869,0.127869
6,No log,0.177338,0.177338
7,No log,0.098896,0.098896
8,No log,0.098705,0.098705
9,No log,0.080939,0.080939
10,0.099000,0.112905,0.112905


TrainOutput(global_step=1080, training_loss=0.06183811955981784, metrics={'train_runtime': 902.714, 'train_samples_per_second': 9.46, 'train_steps_per_second': 1.196, 'total_flos': 0.0, 'train_loss': 0.06183811955981784, 'epoch': 20.0})

**ASAGIDAKI KOD DENEME AMACLIDIR**

In [ ]:
# Örnek metin
sample_text = "Bugün park gitmek için iyi gün. Ben çok mutlu olmak. Orada ağaçlar büyük ve yeşil vardı. Çocuklar oyun oynamak, çok ses yap. Ben oturmak bir bankta, kuş sesleri dinlemek. Bir tane kedi vardı, o bana yakın gelmek. Ben onu bakmak, ama sonra o hızla koşup gitmek. Hava sonra yavaşça karanlık olmak başladı. Ben ev için gitmek karar verdi. Belki yarın gelir, belki ben yok. Park güzel yerdir, ama bazı şeyler tuhaftı. Ne demek ben söyleyemiyorum, ama öyle olmuştu sanırım."

# Tokenizer ile encode et (model eğitilirken kullandığın tokenizer)
inputs = tokenizer(
    sample_text,
    truncation=True,
    padding="max_length",
    max_length=256,
    return_tensors="pt"
)

# Girdi tensörlerini cihaza gönder (GPU/CPU)
inputs = {k: v.to(device) for k, v in inputs.items()}


In [ ]:
model.eval()
inputs = tokenizer(sample_text, return_tensors="pt", padding="max_length", truncation=True, max_length=256)
inputs.pop("token_type_ids", None)  # varsa çıkar


outputs = model(**inputs)


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
print(next(model.parameters()).device)


cuda:0


In [ ]:
for k, v in inputs.items():
    print(k, v.device)


input_ids cpu
attention_mask cpu


In [ ]:
inputs = {k: v.to(device) for k, v in inputs.items()}


In [ ]:
logits = outputs["logits"]  # Modelin çıktısı

# Eğer çıktıyı normalize ettiysen (0-1 arası), burada tekrar orijinal skala’ya çevirebilirsin.
# Örneğin toplam skor 1-10 aralığındaysa:
predicted_score = logits.squeeze().cpu().item()
predicted_score_original_scale = predicted_score * 9 + 1  # 0-1'den 1-10 aralığına dönüşüm

print(f"Tahmin edilen toplam skor (0-1 arası): {predicted_score:.4f}")
print(f"Tahmin edilen toplam skor (1-10 arası): {predicted_score_original_scale:.2f}")


Tahmin edilen toplam skor (0-1 arası): 0.5755
Tahmin edilen toplam skor (1-10 arası): 6.18


In [ ]:
model.eval()

text = "Test etmek istediğin örnek metin buraya."
inputs = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=256)

# Inputs tensörlerini modele uygun cihaza taşı
inputs = {k: v.to(device) for k, v in inputs.items()}

with torch.no_grad():
    outputs = model(**inputs)

# Çıktıyı al
pred = outputs["logits"].squeeze().cpu().numpy()
print("Tahmin sonucu:", pred)

TypeError: BertSingleOutput.forward() got an unexpected keyword argument 'token_type_ids'

**TEST ETMEK ICIN ASAGIDAKI KOD KULLANILACAKTIR**

In [ ]:
import torch


text = " Rüyalar, gece bize gelen gizemli şeyler dir. Bazen çok garip olurlar, anlamsız gelirler insana. Bi keresinde uçtum rüyamda, bulutlara dokundum sanki. Öyle gerçekçiydi ki, uyandığımda kendimi boşlukta hissettim. Rüya görmeyen insan, bişeyleri kaçırıyor bence. Rüya görmek, günlük hayatın stresinden uzaklaşmak gibidir. Geleceyi gösterdiyi söylenir ama ben pek inanmam buna. Her rüya, kendi içimizde saklı kalmış arzuları ortaya çıkarır. Belkide beynimizin kendisi ile konuşmasıdır rüyalar. Bazıları unutulur sabah, bazıları hiç akıldan çıkmaz. Rüya yorumlamak, ayrı bi sanattır."

inputs = tokenizer(
    text,
    truncation=True,
    padding="max_length",
    max_length=256,
    return_tensors="pt",
    return_token_type_ids=False
)

inputs = {k: v.to(device) for k, v in inputs.items()}
model = model.to(device)


model.eval()
with torch.no_grad():
    outputs = model(**inputs)

predicted_score = outputs["logits"].cpu().numpy().flatten()[0]
print(f"Predicted total score: {predicted_score:.4f}")

# Modelin çıktısı 0-1 aralığında ise, orijinal skora çevirmek için:
predicted_score_original_scale = predicted_score * 10

print(f"Predicted total score (orijinal skala 1-10): {predicted_score_original_scale:.2f}")

Predicted total score: 0.6460
Predicted total score (orijinal skala 1-10): 6.46


In [ ]:
torch.save(model.state_dict(), "bert_turkish_model2_state_dict.pth")
tokenizer.save_pretrained("./bert_turkish_tokenizer2")



('./bert_turkish_tokenizer2/tokenizer_config.json',
 './bert_turkish_tokenizer2/special_tokens_map.json',
 './bert_turkish_tokenizer2/vocab.txt',
 './bert_turkish_tokenizer2/added_tokens.json',
 './bert_turkish_tokenizer2/tokenizer.json')

In [ ]:
import shutil
from google.colab import files

shutil.make_archive("bert_turkish_model2", 'zip', root_dir=".", base_dir="bert_turkish_tokenizer2")
files.download("bert_turkish_model2.zip")
files.download("bert_turkish_model2_state_dict.pth")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>